In [16]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
import time

In [17]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model('E:\model.h5')

In [19]:
from twilio.rest import Client



# Your Account SID and Auth Token from twilio.com/console
account_sid ='AC74d458c60a4c56dcf72de55f9a2c6be6'
auth_token ='002573aa7cbd674c9a023c9cdd34f6c7'
client = Client(account_sid, auth_token)


# The phone number to send the SMS to
to_number = '+917085701610'

# Your Twilio phone number
from_number = '+16203093516'

mixer.init()
sound= mixer.Sound(r'E:\Traffic-Road-Accident-Prevention-using-Deep-Learning-\alarm.wav')


cap = cv2.VideoCapture(0)
Score = 0

cv2.setWindowTitle('Eye Detection', 'Eye Detection using Deep Learning')


#For sending msg only once
send_message = False
sent_time = 0




while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    
    #Header Details
    cv2.rectangle(frame, (0, 0), (width, 70), (0, 0, 0), -1)
    cv2.putText(frame, 'Eye Detection', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
   
    cv2.putText(frame, 'Driver Name: Ayush Kumar Sinha', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (160, 32, 240), 2)
    cv2.putText(frame, 'Vehicle Details: XYZ-1234', (10, 135), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (160, 32, 240), 2)

   
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (0,128,0), thickness=3 )
        
    for (ex,ey,ew,eh) in eyes:
        eye= frame[ey:ey+eh,ex:ex+w]
        eye= cv2.resize(eye,(80,80))
        eye= eye/255
        eye= eye.reshape(80,80,3)
        eye= np.expand_dims(eye,axis=0)
        
        prediction = model.predict(eye)
        
        if prediction[0][0]>0.30:
            cv2.putText(frame,'Closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,0,0),
                       thickness=1,lineType=cv2.LINE_AA)
            #Background of Score Text
            cv2.rectangle(frame, (width-200, 10), (width-10, 50), (0, 0, 0), -1)
            cv2.putText(frame,'Score: '+str(Score),(width-150,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,0,0),
                       thickness=1,lineType=cv2.LINE_AA)
            Score=Score+1
            if(Score>15):
                try:
                    sound.play()
                except:
                    pass
                if(Score > 30):
                    #Check if message has already been sent
                    if not send_message:
                        message = client.messages.create(
                            body='Mrr gya',
                            from_=from_number,
                            to=to_number
                        )
                        print('Message sent at', time.strftime('%Y-%m-%d %H:%M:%S'))
                        sent_time = time.time()
                        send_message = True
                    else:
                        # Check if it has been more than 10 seconds since the message was sent
                        if time.time() - sent_time > 10:
                            send_message = False
    
        elif prediction[0][1]>0.90:
            cv2.putText(frame,'Open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),
                       thickness=1,lineType=cv2.LINE_AA) 
            #Background of Score Text
            cv2.rectangle(frame, (width-200, 10), (width-10, 50), (0, 0, 0), -1)
            cv2.putText(frame,'Score: '+str(Score),(width-150,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),
                       thickness=1,lineType=cv2.LINE_AA)
            Score = Score-5
            if (Score<0):
                Score=0
            
    cv2.imshow('Eye Detection',frame)
    cv2.moveWindow('Eye Detection', 500, 200)
    cv2.namedWindow('Eye Detection', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Eye Detection', 800,600)
    
    if cv2.waitKey(2) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
mixer.quit()

1/1 [==============================] - 0s 35ms/step
Message sent at 2023-04-23 00:24:51
1/1 [==============================] - 0s 34ms/step
